In [1]:
#Common tasks libraries.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#Miscellaeneous libraries.
import re
from datetime import timedelta
from joblib import dump, load

#Sklearn libraries to use.
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error

In [2]:
#Make label encoding on multiple columns.
def label_encoder(df,*colsname):
    label_encoding = LabelEncoder()
    for name in colsname:
        newName = "{}_encode".format(name)
        df[newName] = label_encoding.fit_transform(df[name])
        #We changed to one to avoid problems with skforecast
        df[newName] = df[newName] + 1

In [4]:
#Load the training data.
train_df = pd.read_csv('./data/train.csv', parse_dates = ['date'], index_col = 0)
train_group_df = train_df.groupby(by=['family','date','store_nbr']).sum().reset_index()

#join with stores
train_join_df = pd.merge(left = train_group_df, right = stores_df, on = 'store_nbr', how = 'inner')
add_datepart(train_join_df,'date',False)

#Adjust holidays
train_join_df_encode = holidays_on_dataset(train_join_df, holidays_df_new, 
                [['date','city'],'date',['date','city','is_holiday_local','is_holiday_national']],
                [['date','locale_name'],'locale','National'],
                ['family','date','store_nbr', 'sales','onpromotion','city_x','state','type','cluster',
                'Year','Month','Week','Day','Dayofweek','Dayofyear','Is_quarter_end','Is_quarter_start'])

oil_prices_df = pd.read_csv('./data/oil.csv')
oil_prices_df = oil_prices_df.dropna()

train_join_df_encode = pd.merge(left= train_join_df_encode, right=oil_prices_df[['date','dcoilwtico']], on='date', how='left')
train_join_df_encode['oil_price'] = train_join_df_encode['dcoilwtico'].shift().rolling(7, min_periods=1).mean().fillna(train_join_df_encode['dcoilwtico'])
train_join_df_encode.drop(['dcoilwtico'], axis=1, inplace=True)
train_join_df_encode.at[0, 'oil_price'] = 1

#Encode the data.
label_encoder(train_join_df_encode,'city_x','state','type')

In [5]:
#Separate the dataframes and time how much does it takes.
%%time
def separate_dfs(df,col1,col2,col3,cols1):
    df_dict = {}
    cont = 0
    for store in df[col1].unique():
        for fam in df[col2].unique():
            mask_dataset = (df[col2] == fam) & (df[col1] == store)
            new_dataset = df[mask_dataset].copy()
            new_dataset[col3] = pd.to_datetime(new_dataset[col3], format='%Y/%m/%d')
            new_dataset.reset_index(inplace = True)
            new_dataset.set_index(col3, inplace = True, drop = False)
            name = "{}{}".format(fam,store)
            df_dict[name] = new_dataset[cols1]
            cont+=1
            if cont % 100 == 0:
                print(cont)
    return df_dict

df_dict = separate_dfs(train_join_df_encode,'store_nbr','family','date',
                                                    ['date','store_nbr','family_encode','city_x_encode','state_encode',
                                                     'type_encode','cluster_encode','onpromotion',
                                                   'sales','oil_price','is_holiday_national','is_holiday_local'])

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
CPU times: total: 4min 11s
Wall time: 3min 57s


In [6]:
#Save the dictionary of dataframes.
from joblib import dump, load
dump(df_dict,'dataframes_separados.joblib')

['dataframes_separados.joblib']

In [17]:
#Forecast with each dataset using a RandomForestRegressor()
%%time
def forecasting(df_dicts,col1,col2,col3,col4,regressor):
    lst_predictions = []
    lst_errors = []
    lst_models = []
    for key in df_dicts.keys():
        df = df_dict[key]
        limit_train = int(round((len(df['date'].unique()) * 0.8),0))
        train = df.iloc[:limit_train]
        test = df.iloc[limit_train:]
        forecaster = ForecasterAutoreg(regressor = regressor, lags = 365)
        forecaster.fit(y = train[col1], exog = train[[col2,col3,col4]])
        lst_models.append(forecaster)
        steps = len(test)
        predictions = forecaster.predict(steps, exog = test[[col2,col3,col4]])
        lst_predictions.append((predictions))
        error = mean_squared_log_error(
                 y_true = test[col1],
                 y_pred = predictions)
        error = np.sqrt(error)
        lst_errors.append(error)
        print("RMSLE: {} for {}".format(error,key))
    
    
    return lst_predictions, lst_errors, lst_models

        

predictions, errors, models = forecasting(df_dict,'sales','oil_price','is_holiday_national','is_holiday_local',
                                         RandomForestRegressor(random_state=8, n_jobs = -1))


RMSLE: 0.65130530511733 for AUTOMOTIVE1
RMSLE: 0.0 for BABY CARE1
RMSLE: 0.5854982721940329 for BEAUTY1
RMSLE: 0.5333371362805684 for BEVERAGES1
RMSLE: 0.5817203196656121 for BOOKS1
RMSLE: 0.526081892721066 for BREAD/BAKERY1
RMSLE: 0.7700652755009607 for CELEBRATION1
RMSLE: 0.6425760203347429 for CLEANING1
RMSLE: 0.4977067112329315 for DAIRY1
RMSLE: 0.5345812034168723 for DELI1
RMSLE: 0.466446855344609 for EGGS1
RMSLE: 0.7135673746478439 for FROZEN FOODS1
RMSLE: 0.5716199302284873 for GROCERY I1
RMSLE: 0.6181834336149244 for GROCERY II1
RMSLE: 0.6221710509330464 for HARDWARE1
RMSLE: 0.7596985069764223 for HOME AND KITCHEN I1
RMSLE: 0.6502522313747078 for HOME AND KITCHEN II1
RMSLE: 0.43078427206373876 for HOME APPLIANCES1
RMSLE: 0.4922965792232337 for HOME CARE1
RMSLE: 0.5803959511071664 for LADIESWEAR1
RMSLE: 0.8950396309232178 for LAWN AND GARDEN1
RMSLE: 0.9386168629510143 for LINGERIE1
RMSLE: 0.8133769520816392 for LIQUOR,WINE,BEER1
RMSLE: 0.5900262674798171 for MAGAZINES1
RMSLE: 0.

RMSLE: 0.4870694151985539 for POULTRY6
RMSLE: 0.4752905167999969 for PREPARED FOODS6
RMSLE: 0.5545596157207967 for PRODUCE6
RMSLE: 0.849932599877517 for SCHOOL AND OFFICE SUPPLIES6
RMSLE: 0.4571997596379657 for SEAFOOD6
RMSLE: 0.6716817974616671 for AUTOMOTIVE7
RMSLE: 0.2668819979327542 for BABY CARE7
RMSLE: 0.5343059594354164 for BEAUTY7
RMSLE: 0.5191834894992231 for BEVERAGES7
RMSLE: 0.6062460705783432 for BOOKS7
RMSLE: 0.40099115587704215 for BREAD/BAKERY7
RMSLE: 0.5067323849475244 for CELEBRATION7
RMSLE: 0.44620631006781986 for CLEANING7
RMSLE: 0.4142646038354075 for DAIRY7
RMSLE: 0.4060326599559389 for DELI7
RMSLE: 0.34035489199592805 for EGGS7
RMSLE: 1.2933242985752793 for FROZEN FOODS7
RMSLE: 0.4960504381270699 for GROCERY I7
RMSLE: 0.42416572517886225 for GROCERY II7
RMSLE: 0.6400281381696817 for HARDWARE7
RMSLE: 0.6355633822466029 for HOME AND KITCHEN I7
RMSLE: 0.622046694335044 for HOME AND KITCHEN II7
RMSLE: 0.5673258118070361 for HOME APPLIANCES7
RMSLE: 0.4956865380473789 f

RMSLE: 0.9163136581222395 for LIQUOR,WINE,BEER12
RMSLE: 0.6928726174327567 for MAGAZINES12
RMSLE: 0.49657204587186765 for MEATS12
RMSLE: 0.38740746452983693 for PERSONAL CARE12
RMSLE: 0.6749710784543308 for PET SUPPLIES12
RMSLE: 0.5800951701026824 for PLAYERS AND ELECTRONICS12
RMSLE: 0.3525384050697533 for POULTRY12
RMSLE: 0.4519503222823703 for PREPARED FOODS12
RMSLE: 0.3696797806575696 for PRODUCE12
RMSLE: 2.7409501500122557 for SCHOOL AND OFFICE SUPPLIES12
RMSLE: 0.723355383732374 for SEAFOOD12
RMSLE: 0.5628073161879712 for AUTOMOTIVE13
RMSLE: 0.0 for BABY CARE13
RMSLE: 0.7800904019808177 for BEAUTY13
RMSLE: 0.48455424956786314 for BEVERAGES13
RMSLE: 0.0 for BOOKS13
RMSLE: 0.5320539348103652 for BREAD/BAKERY13
RMSLE: 0.7641661909756762 for CELEBRATION13
RMSLE: 0.5025359727401465 for CLEANING13
RMSLE: 0.5002285497765584 for DAIRY13
RMSLE: 0.45665587853080336 for DELI13
RMSLE: 0.7279431721978041 for EGGS13
RMSLE: 2.1607820128598374 for FROZEN FOODS13
RMSLE: 0.4977169516700957 for GROC

RMSLE: 1.4316218885651026 for HOME AND KITCHEN II18
RMSLE: 0.3240076204472129 for HOME APPLIANCES18
RMSLE: 2.9065933936560855 for HOME CARE18
RMSLE: 1.381914660804762 for LADIESWEAR18
RMSLE: 1.1626113232718345 for LAWN AND GARDEN18
RMSLE: 1.1207904350954092 for LINGERIE18
RMSLE: 2.5583270897990595 for LIQUOR,WINE,BEER18
RMSLE: 1.0265966090978884 for MAGAZINES18
RMSLE: 2.36802382544793 for MEATS18
RMSLE: 4.08281601167575 for PERSONAL CARE18
RMSLE: 0.8514149839303204 for PET SUPPLIES18
RMSLE: 1.2820192500932104 for PLAYERS AND ELECTRONICS18
RMSLE: 3.1049319094693755 for POULTRY18
RMSLE: 2.095877238240132 for PREPARED FOODS18
RMSLE: 3.6799435858217207 for PRODUCE18
RMSLE: 1.4921010319153227 for SCHOOL AND OFFICE SUPPLIES18
RMSLE: 1.2464875629056538 for SEAFOOD18
RMSLE: 0.5955128918179048 for AUTOMOTIVE19
RMSLE: 0.617548983070631 for BABY CARE19
RMSLE: 0.7089583304604334 for BEAUTY19
RMSLE: 0.48805479132547996 for BEVERAGES19
RMSLE: 0.0 for BOOKS19
RMSLE: 0.4341481018892791 for BREAD/BAKER

RMSLE: 0.4282612259451522 for CLEANING24
RMSLE: 0.41864050734253344 for DAIRY24
RMSLE: 0.3629716343332998 for DELI24
RMSLE: 0.35976052306675627 for EGGS24
RMSLE: 2.5533529087711933 for FROZEN FOODS24
RMSLE: 0.7582761681071478 for GROCERY I24
RMSLE: 0.7261980056800615 for GROCERY II24
RMSLE: 0.5501922663503153 for HARDWARE24
RMSLE: 0.5327016555154321 for HOME AND KITCHEN I24
RMSLE: 0.6092129631622494 for HOME AND KITCHEN II24
RMSLE: 0.5624974454545759 for HOME APPLIANCES24
RMSLE: 0.38525565340704665 for HOME CARE24
RMSLE: 0.5402683839226247 for LADIESWEAR24
RMSLE: 1.3301476991236914 for LAWN AND GARDEN24
RMSLE: 0.6222143092653851 for LINGERIE24
RMSLE: 1.2690336489256602 for LIQUOR,WINE,BEER24
RMSLE: 0.5532689667641112 for MAGAZINES24
RMSLE: 0.4904517498440102 for MEATS24
RMSLE: 0.3721128928323763 for PERSONAL CARE24
RMSLE: 0.3564009555740372 for PET SUPPLIES24
RMSLE: 0.6340156776400483 for PLAYERS AND ELECTRONICS24
RMSLE: 0.4446519991110134 for POULTRY24
RMSLE: 0.3481597225310076 for PR

RMSLE: 0.6986112492697033 for SEAFOOD29
RMSLE: 0.6526466025786131 for AUTOMOTIVE30
RMSLE: 0.3321676589758443 for BABY CARE30
RMSLE: 0.5767032116937744 for BEAUTY30
RMSLE: 0.46305181138202056 for BEVERAGES30
RMSLE: 0.0 for BOOKS30
RMSLE: 0.4324735483556837 for BREAD/BAKERY30
RMSLE: 0.6961702489135432 for CELEBRATION30
RMSLE: 0.6031686102707063 for CLEANING30
RMSLE: 0.44022468679423404 for DAIRY30
RMSLE: 0.43469771667616797 for DELI30
RMSLE: 0.48765379130211917 for EGGS30
RMSLE: 1.8277692218339558 for FROZEN FOODS30
RMSLE: 0.5166760902694684 for GROCERY I30
RMSLE: 0.8296924275564128 for GROCERY II30
RMSLE: 0.4990610419779077 for HARDWARE30
RMSLE: 0.6284861357217512 for HOME AND KITCHEN I30
RMSLE: 0.6210682124323962 for HOME AND KITCHEN II30
RMSLE: 0.34129962586810486 for HOME APPLIANCES30
RMSLE: 0.5327873768490757 for HOME CARE30
RMSLE: 0.6217166642892972 for LADIESWEAR30
RMSLE: 0.0 for LAWN AND GARDEN30
RMSLE: 0.7503051242850098 for LINGERIE30
RMSLE: 1.060978010669029 for LIQUOR,WINE,BE

RMSLE: 0.5959667047515099 for PLAYERS AND ELECTRONICS35
RMSLE: 0.48707958433801 for POULTRY35
RMSLE: 0.48639865266499976 for PREPARED FOODS35
RMSLE: 0.5007837144881832 for PRODUCE35
RMSLE: 1.0156359808214708 for SCHOOL AND OFFICE SUPPLIES35
RMSLE: 0.6919521451042747 for SEAFOOD35
RMSLE: 0.5935264901731643 for AUTOMOTIVE36
RMSLE: 0.14668145922450357 for BABY CARE36
RMSLE: 0.6486528974445305 for BEAUTY36
RMSLE: 0.5217694667600058 for BEVERAGES36
RMSLE: 0.0 for BOOKS36
RMSLE: 0.44151556833571187 for BREAD/BAKERY36
RMSLE: 0.6635754911292531 for CELEBRATION36
RMSLE: 0.4323776292051505 for CLEANING36
RMSLE: 0.41625495110494753 for DAIRY36
RMSLE: 0.36965904700369695 for DELI36
RMSLE: 0.507549599514536 for EGGS36
RMSLE: 0.8747050123160807 for FROZEN FOODS36
RMSLE: 0.5502600008713859 for GROCERY I36
RMSLE: 0.8898825352491798 for GROCERY II36
RMSLE: 0.4742572205295194 for HARDWARE36
RMSLE: 0.6578082923577863 for HOME AND KITCHEN I36
RMSLE: 0.6564837076218557 for HOME AND KITCHEN II36
RMSLE: 0.38

RMSLE: 0.4812234429341193 for HOME CARE41
RMSLE: 0.7324028270122641 for LADIESWEAR41
RMSLE: 1.1247562474205115 for LAWN AND GARDEN41
RMSLE: 0.736584104884098 for LINGERIE41
RMSLE: 1.7495290484857338 for LIQUOR,WINE,BEER41
RMSLE: 0.5073051866182446 for MAGAZINES41
RMSLE: 0.4476695481811782 for MEATS41
RMSLE: 0.6021854156728388 for PERSONAL CARE41
RMSLE: 0.5725751792654837 for PET SUPPLIES41
RMSLE: 0.574032744079034 for PLAYERS AND ELECTRONICS41
RMSLE: 0.4101359343453792 for POULTRY41
RMSLE: 0.38964602835580764 for PREPARED FOODS41
RMSLE: 0.48012962798711256 for PRODUCE41
RMSLE: 1.1237323309216454 for SCHOOL AND OFFICE SUPPLIES41
RMSLE: 0.5293067992938537 for SEAFOOD41
RMSLE: 0.6803669972143178 for AUTOMOTIVE42
RMSLE: 0.5059008674095796 for BABY CARE42
RMSLE: 0.5991157816181356 for BEAUTY42
RMSLE: 0.5961162413126809 for BEVERAGES42
RMSLE: 0.7155632621548916 for BOOKS42
RMSLE: 0.4159146055348514 for BREAD/BAKERY42
RMSLE: 0.5645278130008021 for CELEBRATION42
RMSLE: 0.4944350264202506 for C

RMSLE: 0.4699005152867457 for EGGS47
RMSLE: 0.6354477618526948 for FROZEN FOODS47
RMSLE: 0.6063834811498304 for GROCERY I47
RMSLE: 0.57799744827424 for GROCERY II47
RMSLE: 0.7329526400040713 for HARDWARE47
RMSLE: 0.6217648857089375 for HOME AND KITCHEN I47
RMSLE: 1.4647953948852452 for HOME AND KITCHEN II47
RMSLE: 0.8023987021998431 for HOME APPLIANCES47
RMSLE: 0.5728729229926648 for HOME CARE47
RMSLE: 0.586001944939831 for LADIESWEAR47
RMSLE: 0.9164419181218927 for LAWN AND GARDEN47
RMSLE: 0.8882167577091302 for LINGERIE47
RMSLE: 1.1092353020947454 for LIQUOR,WINE,BEER47
RMSLE: 0.5131780743432203 for MAGAZINES47
RMSLE: 0.542511882610545 for MEATS47
RMSLE: 0.5491811082607561 for PERSONAL CARE47
RMSLE: 0.4445146009935999 for PET SUPPLIES47
RMSLE: 0.611000011092236 for PLAYERS AND ELECTRONICS47
RMSLE: 0.5299891988242227 for POULTRY47
RMSLE: 0.4460108289216102 for PREPARED FOODS47
RMSLE: 0.551174814746012 for PRODUCE47
RMSLE: 2.9826108127740034 for SCHOOL AND OFFICE SUPPLIES47
RMSLE: 0.46

RMSLE: 0.9404618463297275 for BEAUTY53
RMSLE: 0.6876695151966229 for BEVERAGES53
RMSLE: 0.343469104104566 for BOOKS53
RMSLE: 0.7258896760079866 for BREAD/BAKERY53
RMSLE: 1.0785941125817373 for CELEBRATION53
RMSLE: 1.3409206318447047 for CLEANING53
RMSLE: 0.6737088287620009 for DAIRY53
RMSLE: 0.6902253191170711 for DELI53
RMSLE: 0.6709489734372861 for EGGS53
RMSLE: 0.8653958883611774 for FROZEN FOODS53
RMSLE: 0.688611197703145 for GROCERY I53
RMSLE: 0.775805782584475 for GROCERY II53
RMSLE: 0.8380772205734514 for HARDWARE53
RMSLE: 0.7297952051645477 for HOME AND KITCHEN I53
RMSLE: 0.5549287769704794 for HOME AND KITCHEN II53
RMSLE: 0.2857690502842588 for HOME APPLIANCES53
RMSLE: 1.311698157347166 for HOME CARE53
RMSLE: 0.9670952107074225 for LADIESWEAR53
RMSLE: 0.812376710881849 for LAWN AND GARDEN53
RMSLE: 1.509568685472169 for LINGERIE53
RMSLE: 1.8068546520933142 for LIQUOR,WINE,BEER53
RMSLE: 0.8828783270462272 for MAGAZINES53
RMSLE: 0.5293646936481006 for MEATS53
RMSLE: 2.39002938630

In [19]:
#Save the predictios, errors, and models.
from joblib import dump, load
dump(predictions,'predicts_RF.joblib')
dump(errors,'errors_RF.joblib')
dump(models,'models_RF.joblib')

['models_RF.joblib']

In [20]:
#Load the test dataset
test_data = df_test = pd.read_csv('./data/skforecast_test.csv', index_col = 0)

In [21]:
test_data

,id,family,date,store_nbr,onpromotion,city_x,state,type,cluster,Year,...,Is_quarter_start,is_holiday_local,is_holiday_national,Is_quarter_end_encode,Is_quarter_start_encode,family_encode,city_x_encode,state_encode,type_encode,oil_price
0,3000888,AUTOMOTIVE,2017-08-16,1,0,Quito,Pichincha,D,13,2017,...,False,0.0,0.0,1,1,1,19,13,4,46.800000
1,3002670,AUTOMOTIVE,2017-08-17,1,0,Quito,Pichincha,D,13,2017,...,False,0.0,0.0,1,1,1,19,13,4,46.800000
2,3004452,AUTOMOTIVE,2017-08-18,1,0,Quito,Pichincha,D,13,2017,...,False,0.0,0.0,1,1,1,19,13,4,46.935000
3,3006234,AUTOMOTIVE,2017-08-19,1,0,Quito,Pichincha,D,13,2017,...,False,0.0,0.0,1,1,1,19,13,4,47.486667
4,3008016,AUTOMOTIVE,2017-08-20,1,0,Quito,Pichincha,D,13,2017,...,False,0.0,0.0,1,1,1,19,13,4,47.486667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28507,3022139,SEAFOOD,2017-08-27,54,0,El Carmen,Manabi,C,3,2017,...,False,0.0,0.0,1,1,33,6,11,3,47.676000
28508,3023921,SEAFOOD,2017-08-28,54,0,El Carmen,Manabi,C,3,2017,...,False,0.0,0.0,1,1,33,6,11,3,47.676000
28509,3025703,SEAFOOD,2017-08-29,54,0,El Carmen,Manabi,C,3,2017,...,False,0.0,0.0,1,1,33,6,11,3,47.478000
28510,3027485,SEAFOOD,2017-08-30,54,0,El Carmen,Manabi,C,3,2017,...,False,0.0,0.0,1,1,33,6,11,3,47.240000


In [25]:
#Call the function to store the dataframes on a dictionary.
df_dict_test = separate_dfs(test_data,'store_nbr','family','date',
                                                    ['date','id','store_nbr','family_encode','city_x_encode','state_encode',
                                                     'type_encode','onpromotion',
                                                   'oil_price','is_holiday_national','is_holiday_local'])

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700


In [27]:
#Save the dictionary of datasets.
from joblib import dump, load
dump(df_dict_test,'dataframes_separados_test.joblib')

['dataframes_separados_test.joblib']

In [128]:
#Get the prediction values of each model of each dataset.
%%time
def get_values_of_test_data(dict_df,cols):
    lst_vals = []
    for i, name in enumerate(dict_df.keys()):
        test = dict_df[name].copy()
        steps = len(test)
        predict = models[i].predict(steps = steps, exog = test[cols])
        lst_vals.append(("{}_prediction".format(name),test['id'].values,predict.values))
    
    test_df = pd.DataFrame(data = lst_vals[0][2], index = lst_vals[0][1], columns = ['values'])
    for name, index, values in lst_vals[1:]:
        print("You are joining dataframe: {}".format(name))
        new_piece = pd.DataFrame(data = values, index = index, columns = ['values'])
        test_df = pd.concat(objs=(test_df,new_piece))
    
    return test_df
        
        
        
test_values = get_values_of_test_data(df_dict_test,['oil_price','is_holiday_national','is_holiday_local'])

You are joining dataframe: BABY CARE1_prediction
You are joining dataframe: BEAUTY1_prediction
You are joining dataframe: BEVERAGES1_prediction
You are joining dataframe: BOOKS1_prediction
You are joining dataframe: BREAD/BAKERY1_prediction
You are joining dataframe: CELEBRATION1_prediction
You are joining dataframe: CLEANING1_prediction
You are joining dataframe: DAIRY1_prediction
You are joining dataframe: DELI1_prediction
You are joining dataframe: EGGS1_prediction
You are joining dataframe: FROZEN FOODS1_prediction
You are joining dataframe: GROCERY I1_prediction
You are joining dataframe: GROCERY II1_prediction
You are joining dataframe: HARDWARE1_prediction
You are joining dataframe: HOME AND KITCHEN I1_prediction
You are joining dataframe: HOME AND KITCHEN II1_prediction
You are joining dataframe: HOME APPLIANCES1_prediction
You are joining dataframe: HOME CARE1_prediction
You are joining dataframe: LADIESWEAR1_prediction
You are joining dataframe: LAWN AND GARDEN1_prediction
Yo

You are joining dataframe: BEAUTY12_prediction
You are joining dataframe: BEVERAGES12_prediction
You are joining dataframe: BOOKS12_prediction
You are joining dataframe: BREAD/BAKERY12_prediction
You are joining dataframe: CELEBRATION12_prediction
You are joining dataframe: CLEANING12_prediction
You are joining dataframe: DAIRY12_prediction
You are joining dataframe: DELI12_prediction
You are joining dataframe: EGGS12_prediction
You are joining dataframe: FROZEN FOODS12_prediction
You are joining dataframe: GROCERY I12_prediction
You are joining dataframe: GROCERY II12_prediction
You are joining dataframe: HARDWARE12_prediction
You are joining dataframe: HOME AND KITCHEN I12_prediction
You are joining dataframe: HOME AND KITCHEN II12_prediction
You are joining dataframe: HOME APPLIANCES12_prediction
You are joining dataframe: HOME CARE12_prediction
You are joining dataframe: LADIESWEAR12_prediction
You are joining dataframe: LAWN AND GARDEN12_prediction
You are joining dataframe: LINGE

You are joining dataframe: BREAD/BAKERY22_prediction
You are joining dataframe: CELEBRATION22_prediction
You are joining dataframe: CLEANING22_prediction
You are joining dataframe: DAIRY22_prediction
You are joining dataframe: DELI22_prediction
You are joining dataframe: EGGS22_prediction
You are joining dataframe: FROZEN FOODS22_prediction
You are joining dataframe: GROCERY I22_prediction
You are joining dataframe: GROCERY II22_prediction
You are joining dataframe: HARDWARE22_prediction
You are joining dataframe: HOME AND KITCHEN I22_prediction
You are joining dataframe: HOME AND KITCHEN II22_prediction
You are joining dataframe: HOME APPLIANCES22_prediction
You are joining dataframe: HOME CARE22_prediction
You are joining dataframe: LADIESWEAR22_prediction
You are joining dataframe: LAWN AND GARDEN22_prediction
You are joining dataframe: LINGERIE22_prediction
You are joining dataframe: LIQUOR,WINE,BEER22_prediction
You are joining dataframe: MAGAZINES22_prediction
You are joining dat

You are joining dataframe: EGGS32_prediction
You are joining dataframe: FROZEN FOODS32_prediction
You are joining dataframe: GROCERY I32_prediction
You are joining dataframe: GROCERY II32_prediction
You are joining dataframe: HARDWARE32_prediction
You are joining dataframe: HOME AND KITCHEN I32_prediction
You are joining dataframe: HOME AND KITCHEN II32_prediction
You are joining dataframe: HOME APPLIANCES32_prediction
You are joining dataframe: HOME CARE32_prediction
You are joining dataframe: LADIESWEAR32_prediction
You are joining dataframe: LAWN AND GARDEN32_prediction
You are joining dataframe: LINGERIE32_prediction
You are joining dataframe: LIQUOR,WINE,BEER32_prediction
You are joining dataframe: MAGAZINES32_prediction
You are joining dataframe: MEATS32_prediction
You are joining dataframe: PERSONAL CARE32_prediction
You are joining dataframe: PET SUPPLIES32_prediction
You are joining dataframe: PLAYERS AND ELECTRONICS32_prediction
You are joining dataframe: POULTRY32_prediction

You are joining dataframe: DELI41_prediction
You are joining dataframe: EGGS41_prediction
You are joining dataframe: FROZEN FOODS41_prediction
You are joining dataframe: GROCERY I41_prediction
You are joining dataframe: GROCERY II41_prediction
You are joining dataframe: HARDWARE41_prediction
You are joining dataframe: HOME AND KITCHEN I41_prediction
You are joining dataframe: HOME AND KITCHEN II41_prediction
You are joining dataframe: HOME APPLIANCES41_prediction
You are joining dataframe: HOME CARE41_prediction
You are joining dataframe: LADIESWEAR41_prediction
You are joining dataframe: LAWN AND GARDEN41_prediction
You are joining dataframe: LINGERIE41_prediction
You are joining dataframe: LIQUOR,WINE,BEER41_prediction
You are joining dataframe: MAGAZINES41_prediction
You are joining dataframe: MEATS41_prediction
You are joining dataframe: PERSONAL CARE41_prediction
You are joining dataframe: PET SUPPLIES41_prediction
You are joining dataframe: PLAYERS AND ELECTRONICS41_prediction
Yo

You are joining dataframe: LINGERIE48_prediction
You are joining dataframe: LIQUOR,WINE,BEER48_prediction
You are joining dataframe: MAGAZINES48_prediction
You are joining dataframe: MEATS48_prediction
You are joining dataframe: PERSONAL CARE48_prediction
You are joining dataframe: PET SUPPLIES48_prediction
You are joining dataframe: PLAYERS AND ELECTRONICS48_prediction
You are joining dataframe: POULTRY48_prediction
You are joining dataframe: PREPARED FOODS48_prediction
You are joining dataframe: PRODUCE48_prediction
You are joining dataframe: SCHOOL AND OFFICE SUPPLIES48_prediction
You are joining dataframe: SEAFOOD48_prediction
You are joining dataframe: AUTOMOTIVE49_prediction
You are joining dataframe: BABY CARE49_prediction
You are joining dataframe: BEAUTY49_prediction
You are joining dataframe: BEVERAGES49_prediction
You are joining dataframe: BOOKS49_prediction
You are joining dataframe: BREAD/BAKERY49_prediction
You are joining dataframe: CELEBRATION49_prediction
You are join

In [129]:
#Display the values obtained.
test_values

,values
3000888,2.87
3002670,3.76
3004452,5.52
3006234,5.93
3008016,5.07
...,...
3022139,1.18
3023921,0.96
3025703,1.41
3027485,0.61


In [139]:
#Load the submission data and make it a csv to upload it in kaggle.
submissions = pd.read_csv('./data/sample_submission.csv',index_col = 'id')
submissions['sales'] = test_values['values']
submissions.to_csv('skforecasting_rf.csv')